In [ ]:
# import standard libraries
from autograd import numpy as np
from autograd import grad
from autograd.misc.optimizers import adam, sgd
from autograd import scipy as sp
import autograd.numpy.random as npr
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import sys
import time

# import our libraries
import bayes_helpers as bh
from utils import generate_data, run_toy_nn
from feed_forward import Feedforward
from nlm import NLM
from luna import LUNA

from config import *

### Generate Cubic Dataset

Generates 100 datapoints for train and 100 points for test according to the function

$$y = \frac{1}{2}x^3 + \epsilon$$

$$\epsilon \sim N(0, 3^2)$$

In [ ]:
x_train, y_train, x_test = generate_data()

### Train NLM

In [ ]:
t0 = time.time()
nlm = NLM(prior_variance,y_noise_variance, regularization_param_nlm, nlm_architecture, random_seed)
nlm.train(x_train,y_train, opt_params)
print(f"time: {np.round(time.time() - t0, 3)} seconds")

### Train LUNA

In [ ]:
t0 = time.time()
luna = LUNA(prior_variance, y_noise_variance, regularization_param_luna, similarity_param, luna_architecture, random_seed)
luna.train(x_train, y_train, opt_params)
print(f"time: {np.round(time.time() - t0, 3)} seconds")

### Compare Results
- notice they have identical post processing function calls

In [ ]:
# fig, ax = plt.subplots(1,2, figsize = (12, 6))
i = 0
for name, model in {"Luna":luna,"NLM":nlm}.items():
    posterior_predictions, posterior_predictive_samples = model.predict(x_test)
    
    bh.viz_pp_samples(x_train, y_train,x_test.flatten(),posterior_predictive_samples,name)
    # Compute the 97.5 th percentile of the posterior predictive predictions
#     pp_upper = np.percentile(posterior_predictive_samples, 97.5, axis=0)

#     # Compute the 2.5 th percentile of the posterior predictive predictions
#     pp_lower = np.percentile(posterior_predictive_samples, 2.5, axis=0)

#     # Compute the 50 th percentile of the posterior predictive predictions
#     pp_mean = np.mean(posterior_predictive_samples, axis=0)

#     plt.plot(x_test.flatten(), pp_mean, color='red') # visualize the mean of the posterior predictive
#     plt.fill_between(x_test.flatten(), pp_upper, pp_lower, color='red', alpha=0.4, label='95% Conf. Interval') # visualize the 95% posterior predictive interval
#     plt.scatter(x_train, y_train, color='black', label='training data') # visualize the training data
#     plt.legend()
#     plt.title(name)
#     plt.ylim([-100,100])
#     plt.show()